In [1]:
import numpy as np
import pandas as pd

In [2]:
# pd.read_excel?
df=pd.read_excel('HW cluster data.xlsx',sheet_name=0,
              header=0,
              index_col=0,nrows=8
              )
df#groupby('Clus')

,X,Y,Cluster
case #,,,
1,2,1,1
2,3,2,1
3,4,0,1
4,6,2,2
5,10,4,2
6,4,5,3
7,5,6,3
8,6,4,3


In [3]:
df.loc[:,['cluster_mean_'+str(i) for i in df.columns if i!='Cluster']]=df.groupby('Cluster').transform('mean').values
df

,X,Y,Cluster,cluster_mean_X,cluster_mean_Y
case #,,,,,
1,2,1,1,3.0,1.0
2,3,2,1,3.0,1.0
3,4,0,1,3.0,1.0
4,6,2,2,8.0,3.0
5,10,4,2,8.0,3.0
6,4,5,3,5.0,5.0
7,5,6,3,5.0,5.0
8,6,4,3,5.0,5.0


In [4]:
# values of observation across variables
# mean of variables axis 1

# deviance of variable about mean then their squares
# Finally sum of squared deviances
var_cols=['X','Y']


def within_cluster_sumofSquares(d,var_cols):
  arr=d.loc[:,var_cols].values#shape=obs,var

  cluster_mean_vec=arr.mean(axis=0)#shape=var,
  #cab be directly vectorised/broadcasted


  deviance=arr-cluster_mean_vec
  square_deviance=pow(deviance,2)
  sum_square_deviance=square_deviance.sum().sum()#first summ across each variable for all observation and then sum of deviance across each variable
  # print(deviance)
  return sum_square_deviance

df_cluster_sumofSquares=df.groupby(['Cluster']).apply(within_cluster_sumofSquares,var_cols=var_cols)
df_cluster_sumofSquares



Cluster
1     4.0
2    10.0
3     4.0
dtype: float64

In [5]:
wss=df_cluster_sumofSquares.sum()
wss

18.0

# BSS
bss is primarily associated with deviance of cluster mean vector from the global mean vector

The computation has to also consider the number of data points within each cluster


-- Compute local mean and global means


>   Create a l2 deviance vector for the distance between two means

>   Create count vector for each cluster


>> Compute product of two and then vector sum to generate BSS



In [6]:
global_mean_vector=df.loc[:,var_cols].mean().values#shape var,

cluster_mean_vector=df.groupby('Cluster')[var_cols].mean().values# cluster,var
cluster_count=df.Cluster.value_counts(sort=False).values# cluster,

#lets compute the l2 diviance first so that the vector shape gets aligned

l2_deviance=np.sum(np.power(cluster_mean_vector-global_mean_vector,2),axis=1)#we need square deviance for each cluster computed across all the variables
l2_deviance

weighted_l2Deviance=l2_deviance*cluster_count
weighted_l2Deviance

array([24., 18., 12.])

In [7]:
bss=sum(weighted_l2Deviance)
bss

54.0

# TSS
We will verify TSS= BSS +Wss

TSS is the deviance across global_mean
We can consider the entire dataset as the global cluster and computer square of deviance

In [8]:
tss=within_cluster_sumofSquares(df,var_cols)
tss

72.0

# R2
R square determines how much variation is being explained by the new group of means as compared to the previous ones

Previous one in generalised case refers to the Global mean 

r2=1-(WSS/TSS)
>=BSS/TSS


In [9]:
r2=bss/tss
r2

0.75

# If each data point was itself a cluster

In [10]:
# If each data point was itself a cluster then the WSS would have been 0 and no variation is left to be explained when we have considered all the data points
#BSS would have been equal to TSS and r2 thus be 1
BSS=TSS=72
WSS=0

# IF we only had one cluster 

In [11]:
# TSS=same as before as it is a holistic data property
# WSS=TSS as we only have one cluster
# BSS=0 as there is only one local mean that is the global mean 
#Thus r2 would have been 0 (BSS/TSS)